In [2]:
import os
import sys
app_path = os.path.abspath('..')
sys.path.insert(0, app_path)

from datetime import datetime, UTC
from dotenv import load_dotenv

from langchain import PromptTemplate
from langchain_ollama import ChatOllama

from app.chroma_client import get_embeddings, get_client

load_dotenv()

CHANNEL_NAME = os.getenv('TELEGRAM_CHAT_NAME')
CHANNEL_ID = os.getenv('TELEGRAM_CHAT_ID')
try:
    CHANNEL_ID = int(CHANNEL_ID)
except:
    pass

print(f'Channel name is {CHANNEL_NAME}')
print(f'Channel id is {CHANNEL_ID}')

Channel name is go_to_vilnius_e5
Channel id is -1133953167


In [4]:
# ru_model_name = "cointegrated/rubert-tiny2"
ru_model_name = "intfloat/multilingual-e5-large-instruct"
embeddings = get_embeddings(ru_model_name)

chroma_client_from_telegram = get_client(f'telegram_{CHANNEL_NAME}', embeddings)

In [5]:
llm_qwen3_8b = ChatOllama(
    model="qwen3:8b",
)

In [38]:
user_query = "Выведи самые токсичные сообщения в чате."
user_query = "Выведи самые позитивные сообщения в чате."
user_query = "Выведи самые опасные места в Вильнюсе."
# user_query = "Можно ли компенсировать отель или аквапарк через велнес/БТА/бта/BTA?"
# user_query = "Какие лучшие бани или сауны?"
# user_query = "Как получить детские деньги, компенсацию по уходу за ребенком."

In [39]:
ru_telegram_prompt_expanding = PromptTemplate(
    input_variables=["user_query", "n"],
    template="""
    Напиши {n} гипотетических ответов на запрос пользователя.
    Не добавляй заголовков, авторов и другой дополнительной информации к твоим ответам.

    Вопрос пользователя:
    {user_query}

    Гипотетические ответы:
    """,
)

ru_telegram_prompt_expanding = PromptTemplate(
    input_variables=["user_query", "n"],
    template="""
Контекст данных:
  – Пользовательский запрос: {user_query}

Задача:
  Сгенерировать ровно {n} гипотетических, но конкретных и практически релевантных ответов на основе вышеуказанного запроса от пользователя.

Требования к каждому ответу:
  1. Длина не более 2–3 предложений.
  2. Формат выходных данных: маркированный.
  3. Не добавлять заголовки, авторов, ссылки или другие метаданные.

Пример:
  Вопрос: «Какие самые позитивные сообщения в чате?»
  Ответ:
  - Ого, какая замечательная идея! Это точно поднимет настроение всем!
  - Спасибо за вашу помощь — вы просто спасли мой день!
  - Мне очень нравится, как вы подходите к решению задач: вдохновляет!
  - Ваше сообщение заставило меня улыбнуться — вы лучик света в этом чате!
""",
)

hyde_chain_r1_8b = ru_telegram_prompt_expanding | llm_qwen3_8b
raw_hypothetical_answers = hyde_chain_r1_8b.invoke({"user_query": user_query, "n": 15}).content
raw_hypothetical_answers

'<think>\nХорошо, пользователь просит вывести самые опасные места в Вильнюсе. Надо сгенерировать 15 гипотетических, но конкретных и релевантных ответов. Каждый ответ должен быть 2-3 предложения, маркированный, без заголовков и метаданных.\n\nСначала подумаю о возможных опасных местах в Вильнюсе. Например, темные улицы в центре, старые здания, места с высокой кriminalностью. Нужно учесть, что Вильнюс — столица Литвы, относительно безопасный город, но все равно есть зоны с риском.\n\nМожет, упомянуть районы, где часто происходят преступления, например, Калварийская гора, район у реки, старые кварталы. Также стоит добавить советы по безопасности, например, избегать одиночных прогулок, проверять окружение.\n\nНадо убедиться, что каждый ответ уникален и конкретен. Нужно избегать повторений и охватить разные районы. Также важно сохранять нейтральный тон, не обвинять конкретные места, а просто указывать возможные риски.\n\nПроверю, чтобы каждый ответ соответствовал требованиям: 2-3 предложени

In [40]:
hypothetical_answers = raw_hypothetical_answers.split('</think>', maxsplit=1)[-1]
user_query_expanded = f'{user_query}\n\n{hypothetical_answers}'

In [41]:
query = f"Instruct: Given a question, retrieve relevant documents that best answer the question\nQuery: {user_query_expanded}"
related_docs = chroma_client_from_telegram.similarity_search_with_relevance_scores(query, k=15)

t_link_base = "https://web.telegram.org/k/#?tgaddr=tg%3A%2F%2Fprivatepost%3Fchannel%3D1133953167%26post%3D"
print("---".join(f"""
{score} {datetime.fromtimestamp(doc.metadata['date'], UTC)} {t_link_base}{doc.metadata['id']} - {doc.metadata['sender_name']}: {doc.page_content}
""" for doc, score in related_docs))


0.7988540390412127 2025-02-13 21:38:04+00:00 https://web.telegram.org/k/#?tgaddr=tg%3A%2F%2Fprivatepost%3Fchannel%3D1133953167%26post%3D112773 - terra_lncognita: ☝️ **Внимание: загрязнение воздуха в Вильнюсе и других городах Литвы достигло опасных уровней**

Вечером 13 февраля в Вильнюсе уровень загрязнения воздуха твердыми частицами (PM2.5) резко возрос и [достиг](https://www.iqair.com/lithuania/vilnius?srsltid=AfmBOop5mbvtD-V0l-Yb3f-EPVg363H-5zMCga3y8QuFdW5AQyXrxC6B) отметки 117 по индексу AQI, что считается вредным для чувствительных групп населения. 

Блогер Олег Сомин [написал](https://www.facebook.com/share/p/14kz7VtPbX/?mibextid=wwXIfr) в фейсбуке, что его очиститель воздуха зафиксировал значительное превышение нормы, а данные на сайте IQAir это подтвердили.

Твердые частицы PM2.5 — это микроскопические вещества, которые проникают глубоко в легкие, вызывая респираторные и сердечно-сосудистые заболевания. Особую опасность они представляют для детей, пожилых людей и людей с хрони

In [42]:
filtered_related_docs = filter(lambda doc_score: doc_score[-1] > 0.05, related_docs)

context = "---".join(f"""
{datetime.fromtimestamp(doc.metadata['date'], UTC)} - {doc.page_content}
""" for doc, _score in filtered_related_docs)

In [43]:
ru_telegram_prompt = PromptTemplate(
    input_variables=["context", "user_query"],
    template="""
    Ты полезный AI ассистент, который отвечает на вопросы пользователя на основе контекста.
    Контекст это релевантные вопросу сообщения из телеграм чата.

    Сообщения из чата:
    {context}

    Вопрос пользователя:
    {user_query}

    Если в контексте недостаточно информации, чтобы ответить на вопрос пользователя, то скажи, что недостаточно информации.

    Ответ:
    """,
)

hyde_chain_r1_8b = ru_telegram_prompt | llm_qwen3_8b
llm_response = hyde_chain_r1_8b.invoke({"user_query": user_query, "context": context}).content
print(llm_response)

<think>
Хорошо, пользователь спрашивает о самых опасных местах в Вильнюсе. Посмотрю на предоставленные данные. В контексте упоминаются различные события, включая рейды полиции, туннель под проспектом Конституции, автомобиль с советской символикой, и некоторые инциденты с полицией. Однако, везде говорится о действиях властей, проверках, или инцидентах, но нет конкретной информации о преступлениях, кражах, нападениях или других опасных местах. В текстах упоминаются полицейские рейды, что может указывать на безопасность, но не на опасные районы. Также есть информация о том, что в некоторых районах, возможно, есть проблемы с транспортом или строительством, но это не преступность. Нет данных о преступности, кражах, нападениях, или других опасностях. Поэтому, похоже, что в предоставленном контексте недостаточно информации для ответа на вопрос о самых опасных местах.
</think>

В предоставленном контексте недостаточно информации для определения самых опасных мест в Вильнюсе. В текстах упоминаю